# CS 109A/AC 209A/STAT 121A Data Science: Final Project
**Harvard University**<br>
**Fall 2016**<br>
**Instructors:** W. Pan, P. Protopapas, K. Rader<br>
**Members: ** Shawn Pan, Xinyuan (Amy) Wang, Ming-long Wu

## Remark - 051_linear_regressoin.ipynb ##
- Do linear regression models


In [2]:
import time
import pandas as pd
import numpy as np
import scipy as sp
from scipy.signal import savgol_filter
from scipy.stats import mode
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import dates
from matplotlib import colors
import matplotlib.dates as mdates
from datetime import datetime
from collections import Counter
from mpl_toolkits.basemap import Basemap
import statsmodels.api as sm
from sklearn import preprocessing
from collections import Counter
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.feature_selection import RFE
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
%matplotlib inline

### Load data

In [3]:
# Load listing data
listing = pd.read_csv('../../Data/data_after_predictor_selection.csv')
print(listing.columns.values)
listing.head()

['accommodates' 'bathrooms' 'bedrooms' 'review_scores_checkin'
 'review_scores_communication' 'latitude' 'longitude' 'property_type_0'
 'property_type_1' 'property_type_2' 'property_type_3' 'property_type_4'
 'property_type_5' 'room_type_0' 'room_type_1' 'room_type_2' 'bed_type_0'
 'bed_type_1' 'bed_type_2' 'bed_type_3' 'bed_type_4' 'beds' 'price']


,accommodates,bathrooms,bedrooms,review_scores_checkin,review_scores_communication,latitude,longitude,property_type_0,property_type_1,property_type_2,...,room_type_0,room_type_1,room_type_2,bed_type_0,bed_type_1,bed_type_2,bed_type_3,bed_type_4,beds,price
0,2,1.0,1.0,9.0,9.0,40.756852,-73.964754,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,160
1,10,1.0,3.0,9.0,8.0,40.830599,-73.941014,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,105
2,2,1.0,1.0,10.0,10.0,40.692189,-73.924120,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,58
3,2,1.0,1.0,10.0,10.0,40.734751,-74.002592,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,185
4,2,1.0,1.0,10.0,10.0,40.745282,-73.997836,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,195


## Feature engineering: location clustering to price by KNN ##

In [4]:
#Step to convert longitude and latitude to price cluster by KNN
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from sklearn.neighbors import KNeighborsRegressor

class LocationPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.model = KNeighborsRegressor(n_neighbors=50, weights="uniform")
        super(LocationPreprocessor, self).__init__()
        
    def fit_transform(self, X, y):
        self.model.fit(X[["longitude", "latitude"]], y)
        return self.transform(X)
    
    def transform(self, X):
        loc = self.model.predict(X[["longitude", "latitude"]])
        Xnew = X.copy()
        Xnew["location"] = loc
        return Xnew

## Split data for training and testing data with transformation log(y)##

In [7]:
# Get x and y
x = listing
x = x.drop('price', axis=1)
y = listing['price']
print x.shape
print y.shape

(27392, 22)
(27392,)


In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, np.log(y), test_size=0.3, random_state=0)

## Linear Models: ##
### Linear regression ###

In [27]:
#fit linear regression model
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

model = Pipeline([
            ("location", LocationPreprocessor()),
            ("normalize", StandardScaler()),
            ("LinearModel", LinearRegression())
        ])

model.fit(x_train, y_train)
print "Train Score", model.score(x_train, y_train)
print "Test Score", model.score(x_test, y_test)
LR_score = pd.Series([model.score(x_train, y_train), model.score(x_train, y_train)], 
                                                index=['Train R^2', 'Test R^2'])

Train Score 0.659156322163
Test Score 0.658939280446


### Lasso regression (L1 regularization) ###

In [28]:
#fit Lasso model
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso

model = Pipeline([
            ("location", LocationPreprocessor()),
            ("normalize", StandardScaler()),
            ("lasso", Lasso(alpha=0.02))
        ])

model.fit(x_train, y_train)
print "Train Score", model.score(x_train, y_train)
print "Test Score", model.score(x_test, y_test)
Lasso_score = pd.Series([model.score(x_train, y_train), model.score(x_train, y_train)], 
                                                index=['Train R^2', 'Test R^2'])

Train Score 0.649100864543
Test Score 0.647159495484


### Ridge regression (L2 regularization) ###

In [29]:
#fit Ridge model
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge

model = Pipeline([
            ("location", LocationPreprocessor()),
            ("normalize", StandardScaler()),
            ("ridge", Ridge())
        ])

model.fit(x_train, y_train)
print "Train Score", model.score(x_train, y_train)
print "Test Score", model.score(x_test, y_test)
Ridge_score = pd.Series([model.score(x_train, y_train), model.score(x_train, y_train)], 
                                                index=['Train R^2', 'Test R^2'])

Train Score 0.659166667063
Test Score 0.659133228189


### ElasticNet regression (L1 + L2 regularization) ###

In [30]:
#fit Ridge model
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet

model = Pipeline([
            ("location", LocationPreprocessor()),
            ("normalize", StandardScaler()),
            ("elasticnet", ElasticNet (alpha=0.02))
        ])

model.fit(x_train, y_train)
print "Train Score", model.score(x_train, y_train)
print "Test Score", model.score(x_test, y_test)
ElasticNet_score = pd.Series([model.score(x_train, y_train), model.score(x_train, y_train)], 
                                                index=['Train R^2', 'Test R^2'])

Train Score 0.654125161935
Test Score 0.653142586191


### Compare four linear models ###

In [31]:
#compare all scores
score_df = pd.DataFrame({'linear regression': LR_score, 
                         'Lasso': Lasso_score,
                         'Ridge': Ridge_score,
                         'Elastic Net': ElasticNet_score})
score_df

,Elastic Net,Lasso,Ridge,linear regression
Train R^2,0.654125,0.649101,0.659167,0.659156
Test R^2,0.654125,0.649101,0.659167,0.659156


### Conclusion: four linear models don't differ a lot in performance ###